In [1]:
import os
import shutil
from tqdm import tqdm
import cv2
import gc
import pandas as pd
gc.enable()

In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.6
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [3]:
data_gen_args = dict(rotation_range=0.2,
                     rescale=1./255,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

In [4]:
from dataFlatten import trainGenerator, validationGenerator
train_generator = trainGenerator(4, 'data/train/image', pd.read_csv('train.csv'), 
                           data_gen_args, save_to_dir="data/train/aug", image_color_mode="rgb",
                           target_size=(1024,128))

validation_generator = validationGenerator(2, 'data/val/image', pd.read_csv('val.csv'), 
                                           data_gen_args, save_to_dir="data/val/aug", image_color_mode="rgb",
                                           target_size=(1024,128))


Found 1782 validated image filenames.
Found 209 validated image filenames.


In [3]:
from vgg_model import vgg_16, vgg_12, vgg_9
#model = vgg12(pretrained_weights='vgg12.hdf5', input_size=(1024,128,3))
model = vgg12(input_size=(1024,128,3))

In [1]:
from vgg_regressor import ModelCheckpoint, EarlyStopping
early_stopping = EarlyStopping(monitor='val_mean_squared_error', mode='min', patience=15, verbose=1)
model_checkpoint = ModelCheckpoint('vgg12.hdf5', monitor='val_mean_squared_error', verbose=1, save_best_only=True)

C:\Users\abhinav.jhanwar\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
history = model.fit_generator(train_generator, 
                              epochs=50,
                              steps_per_epoch=445,
                              validation_data=validation_generator,
                              validation_steps=104,
                              callbacks=[model_checkpoint, early_stopping],
                              verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
119/446 [=======>......................] - ETA: 2:46 - loss: 0.1224 - mean_squared_error: 0.1224 - mean_absolute_error: 0.2324 - mean_absolute_percentage_error: 9.6331

# Predictions

In [ ]:
from tensorflow.python.keras.models import load_model
model = load_model('vgg12.hdf5')

In [ ]:
from dataFlatten import testGenerator
import pandas as pd
# define test generator for preprocessing data as per train data
data_gen_args = dict(#rotation_range=0.2,
                    #width_shift_range=0.05,
                    #height_shift_range=0.05,
                    #shear_range=0.05,
                    #zoom_range=0.05,
                    #horizontal_flip=True,
                    #fill_mode='nearest',
                    rescale=1./255
)

test_generator = testGenerator(2, 'data/test/image', pd.read_csv('test.csv'),
                               data_gen_args, x_col='field', y_col='lai', 
                               save_to_dir="data/test/aug", image_color_mode="rgb",
                               target_size=(1152,192))

# load test data csv file
test_data = pd.read_csv('test.csv')

# make perdictions for the test data
predictions = model.predict_generator(test_generator)

# save predictions in the csv file
test_data['lai_pred'] = predictions
test_data.to_csv('test_data_pred.csv')

In [ ]:
import numpy as np
from sklearn import metrics
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# print various perfomance metrics for the test data
print("RMSE: ", np.sqrt(metrics.mean_squared_error(test_data['lai'].values, test_data.lai_pred)))
print("MSE: ", metrics.mean_squared_error(test_data['lai'].values, test_data.lai_pred))
print("MAE: ", metrics.mean_absolute_error(test_data['lai'].values, test_data.lai_pred))
print("MAPE: ", mean_absolute_percentage_error(test_data.lai, test_data.lai_pred))